In [1]:
!pip install deep-phonemizer

You should consider upgrading via the '/home/santiago/pytorch/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import glob
import matplotlib.pyplot as plt

allFiles = glob.glob("dp/notebooks/lexicons/*")
allDF = (pd.read_csv(f, encoding='utf-8', sep='\t', names=['grapheme', 'phoneme']) for f in allFiles)

df = pd.concat(allDF, ignore_index=True)
df.insert(0, 'lang', 'pt_br')

2021-12-01 00:43:47,503.503 DEBUG __init__:  (private) matplotlib data path: /home/santiago/pytorch/lib/python3.6/site-packages/matplotlib/mpl-data
2021-12-01 00:43:47,505.505 DEBUG __init__:  matplotlib data path: /home/santiago/pytorch/lib/python3.6/site-packages/matplotlib/mpl-data
2021-12-01 00:43:47,519.519 DEBUG __init__:  CONFIGDIR=/home/santiago/.config/matplotlib
2021-12-01 00:43:47,526.526 DEBUG __init__:  matplotlib version 3.3.4
2021-12-01 00:43:47,527.527 DEBUG __init__:  interactive is False
2021-12-01 00:43:47,529.529 DEBUG __init__:  platform is linux
2021-12-01 00:43:47,532.532 DEBUG __init__:  loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpat

In [5]:
df['grapheme'] = df['grapheme'].map(str)
df['phoneme'] = df['phoneme'].map(str)

#df['phoneme'] = df['phoneme'].str.replace('.', '')
#df['phoneme'] = df['phoneme'].str.replace("'", '')

In [6]:
graphemes = ''.join(sorted(list(set(df['grapheme'].sum()))))
phonemes = sorted(list(set(df['phoneme'].str.split("\\").sum())))[3:]
phonemes.append(" ")
phonemes.append("~")

In [7]:
df['phoneme'] = df['phoneme'].str.replace('\\', '')
df['gsize'] = df['grapheme'].apply(lambda x : len(x))
df['psize'] = df['phoneme'].apply(lambda x : len(x))
gsizemax = df['gsize'].max()

In [14]:
df[df['gsize'] > 225]['grapheme'][196]

'Inspirado por uma experiência bem-sucedida nas comemorações do primeiro centenário da Proclamação da República, Tales encomendou-me uma série de especiais históricos, que seriam distribuídos com a edição regular de Veja como brinde para seus assinantes e compradores de banca.'

In [7]:
subset = df[['lang', 'grapheme', 'phoneme']]
train_data = [tuple(x) for x in subset.to_numpy()]

In [8]:
len(train_data)

10180

In [24]:
# Read standard config and adjust some params for speedup
from dp.utils.io import read_config, save_config
import dp
import os

config_file = os.path.dirname(dp.__file__) + '/configs/forward_config.yaml'
config = read_config(config_file)
config['preprocessing']['languages'] = ['pt_br']
config['preprocessing']['text_symbols'] = graphemes
config['preprocessing']['phoneme_symbols'] = phonemes
config['preprocessing']['lowercase'] = False
config['preprocessing']['n_val'] = 2000

config['training']['epochs'] = 10
config['training']['batch_size'] = 8
config['training']['batch_size_val'] = 8
config['training']['warmup_steps'] = 100
config['training']['generate_steps'] = 1000
config['training']['validate_steps'] = 1000
config['training']['checkpoint_steps'] = 2000
save_config(config, 'config.yaml')

for k, v in config.items():
	print(f'{k} {v}')

paths {'checkpoint_dir': 'checkpoints', 'data_dir': 'datasets'}
preprocessing {'languages': ['pt_br'], 'text_symbols': ' %+,0123456789=abcdefghijklmnopqrstuvwxyzªàáâãçèéêíîïóôõöúûüý', 'phoneme_symbols': ["'", '.', 'E', 'J', 'L', 'O', 'R', 'S', 'X', 'Z', 'a', 'a~', 'b', 'd', 'dZ', 'e', 'ej', 'e~', 'e~j~', 'f', 'g', 'i', 'i~', 'j', 'js', 'j~', 'j~s', 'k', 'l', 'm', 'n', 'o', 'ow', 'o~', 'p', 'pau', 'r', 's', 't', 'tS', 'u', 'u~', 'v', 'w', 'w~', 'z', ' ', '~'], 'char_repeats': 3, 'lowercase': False, 'n_val': 2000}
model {'type': 'transformer', 'd_model': 512, 'd_fft': 1024, 'layers': 6, 'dropout': 0.1, 'heads': 4}
training {'learning_rate': 0.0001, 'warmup_steps': 100, 'scheduler_plateau_factor': 0.5, 'scheduler_plateau_patience': 10, 'batch_size': 8, 'batch_size_val': 8, 'epochs': 10, 'generate_steps': 1000, 'validate_steps': 1000, 'checkpoint_steps': 2000, 'n_generate_samples': 10, 'store_phoneme_dict_in_model': True}


In [25]:
from dp.preprocess import preprocess
from dp.train import train
import torch

torch.cuda.empty_cache()
preprocess(config_file='config.yaml', train_data=train_data, deduplicate_train_data=False)
train(config_file='config.yaml')

2021-11-06 22:29:36,366.366 INFO preprocess:  Preprocessing, train data: with 10180 files.
2021-11-06 22:29:36,388.388 INFO preprocess:  Performing random split with num val: 2000
2021-11-06 22:29:36,402.402 INFO preprocess:  Processing train data...
100%|██████████| 8154/8154 [00:00<00:00, 11334.77it/s]
2021-11-06 22:29:37,272.272 INFO preprocess:  
Saving datasets to: /home/santiago/DeepPhonemizer-main/dp/notebooks/datasets
2021-11-06 22:29:37,420.420 INFO preprocess:  Preprocessing. 
Train counts (deduplicated): [('pt_br', 8154)]
Val counts (including duplicates): [('pt_br', 2026)]
2021-11-06 22:29:37,437.437 INFO train:  Initializing new model from config...
2021-11-06 22:29:37,470.470 INFO train:  Checkpoints will be stored at /home/santiago/DeepPhonemizer-main/dp/notebooks/checkpoints
Epoch: 1 | Step 1019 | Loss: 1.487: 100%|██████████| 1019/1019 [02:55<00:00,  5.81it/s]
Epoch: 2 | Step 2038 | Loss: 0.9793: 100%|██████████| 1019/1019 [03:33<00:00,  4.77it/s]
Epoch: 3 | Step 3057 

In [16]:
# Load phonemizer (including the training data dictionary)
from dp.phonemizer import Phonemizer

phonemizer = Phonemizer.from_checkpoint(
    './checkpoints/autoreg/batch-8/model-512/fft-1024/best_model.pt')
result = phonemizer('Inspirado por uma experiência bem-sucedida nas comemorações do primeiro centenário da Proclamação da República, Tales encomendou-me uma série de especiais históricos, que seriam distribuídos com a edição regular de Veja como brinde para seus assinantes e compradores de banca.', lang='pt_br')

print(result)

2021-12-01 00:46:25,899.899 DEBUG phonemizer:  Initializing phonemizer with model step 53618


i~s.pi'ra.du poX u~.ma es.pe.ri'e~.sia be~j~-su.se'dZi.da najs ko.me.mo.ra'so~j~s du pri~'mej.ru se~.te'na.riu da pro.kla.ma'sa~w~ da Re'publi.ka, 'ta.lis e~.ko.me~'dow-'mi u~.ma 'sEri dZi es.pe.si'ajs is'tOri.kus, ki se'ri.a~w~ dZis.tri.bu'idus ko~ 'a e.dZi'sa~w~ Re.gu'laX dZi 've.Za 'ko~.mu bri~'dZi 'pa.ra 'se.us a.si~'na~.tSis 'ejs ko~.pra'do.ris dZi 'ba~.ka.


In [3]:
# Phonemize a list of texts and pull out model predictions with confidence scores
result = phonemizer.phonemise_list(['carregar ela'], lang='pt_br')

for word, pred in result.predictions.items():
	print(f'{word} {pred.phonemes} {pred.confidence}')

carregar ka.Re'gaX 0.9999020113912649
ela 'E.la 0.9998041492615111
